
# ДЗ 1 — Уртенов  


Выполненные шаги:
1. Загрузка и очистка данных  
2. Создание гео- и временных признаков  
3. One-Hot Encoding и стандартизация  
4. KNNRegressor (ручной и sklearn)  
5. Линейная регрессия с оптимизаторами (SGD, Momentum, AdaGrad) + сравнение со sklearn  
6. Графики функции потерь  
7. Сводная таблица метрик и выводы


In [1]:

# Указываем путь к данным, seed для воспроизводимости и другие гиперпараметры
DATA_PATH = "/content/input_data.csv"
RANDOM_STATE = 42
TEST_SIZE = 0.2
K_NEIGHBORS = 7
CITY_RADIUS_KM = 20


In [2]:

# 📦 Импорт библиотек
import os, math, warnings
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor as SKKNN
from sklearn.linear_model import LinearRegression as SKLR, SGDRegressor

import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Проверим наличие файла
assert os.path.exists(DATA_PATH), f"Файл не найден: {DATA_PATH}"


## 🧹 Шаг 1 — Загрузка и базовая очистка данных

In [3]:

# Читаем CSV с диска
df_raw = pd.read_csv(DATA_PATH)

# Определим ключевые столбцы
name_map = {
    'price': ['price','target','y','cost','sale_price'],
    'date': ['date','dt','created_at','timestamp','publish_date'],
    'geo_lat': ['geo_lat','lat','latitude','y_coord'],
    'geo_lon': ['geo_lon','lon','long','lng','longitude','x_coord'],
}
def find_col(df, cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

price_col = find_col(df_raw, name_map['price'])
date_col  = find_col(df_raw, name_map['date'])
lat_col   = find_col(df_raw, name_map['geo_lat'])
lon_col   = find_col(df_raw, name_map['geo_lon'])

df = df_raw.copy()

# Преобразуем даты, фильтруем пропуски и невалидные координаты
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
if price_col:
    df = df[df[price_col].notnull()]
if lat_col and lon_col:
    df = df[(df[lat_col].between(-90,90)) & (df[lon_col].between(-180,180))]

df = df.reset_index(drop=True)
print("Размер до очистки:", df_raw.shape, "| после:", df.shape)
df.head()


Размер до очистки: (807851, 1) | после: (807851, 1)


,date;price;level;levels;rooms;area;kitchen_area;geo_lat;geo_lon;building_type;object_type;postal_code;street_id;id_region;house_id
0,2021-01-01;2451300;15;31;1;30.3;0;56.7801124;6...
1,2021-01-01;1450000;5;5;1;33;6;44.6081542;40.13...
2,2021-01-01;10700000;4;13;3;85;12;55.5400601;37...
3,2021-01-01;3100000;3;5;3;82;9;44.6081542;40.13...
4,2021-01-01;2500000;2;3;1;30;9;44.7386846;37.71...


## 🌍 Шаг 2 — Гео- и временные признаки

In [4]:

MOSCOW = (55.7558, 37.6176)
SPB    = (59.9343, 30.3351)
EARTH_R = 6371.0

# Функция для расчета расстояния
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arcsin(np.sqrt(a))
    return EARTH_R * c

# Добавляем признаки расстояния и флаги близости к центрам городов
if lat_col and lon_col:
    df['dist_moscow_km'] = haversine(df[lat_col].values, df[lon_col].values, MOSCOW[0], MOSCOW[1])
    df['dist_spb_km']    = haversine(df[lat_col].values, df[lon_col].values, SPB[0], SPB[1])
    df['is_Moscow'] = (df['dist_moscow_km'] <= CITY_RADIUS_KM).astype(int)
    df['is_Saint_Peterburg'] = (df['dist_spb_km'] <= CITY_RADIUS_KM).astype(int)
else:
    df['dist_moscow_km'] = df['dist_spb_km'] = 0
    df['is_Moscow'] = df['is_Saint_Peterburg'] = 0

# Временные признаки
if date_col and pd.api.types.is_datetime64_any_dtype(df[date_col]):
    ref_date = df[date_col].min()
    df['days_since'] = (df[date_col] - ref_date).dt.days
    df['month'] = df[date_col].dt.month
    df['weekday'] = df[date_col].dt.weekday
else:
    df['days_since'] = df['month'] = df['weekday'] = 0

df[['dist_moscow_km','dist_spb_km','is_Moscow','is_Saint_Peterburg','days_since','month','weekday']].head()


,dist_moscow_km,dist_spb_km,is_Moscow,is_Saint_Peterburg,days_since,month,weekday
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


## ⚙️ Шаг 3 — One-Hot Encoding и стандартизация

In [ ]:

# Категориальные и числовые признаки
cat_cols = [c for c in df.columns if (df[c].dtype=='object' or str(df[c].dtype)=='category') and c not in [price_col, date_col]]
num_cols = [c for c in df.columns if c not in cat_cols + [price_col, date_col]]

# One-hot кодирование категориальных признаков
df_cat = pd.get_dummies(df[cat_cols], drop_first=True) if len(cat_cols) else pd.DataFrame(index=df.index)
df_num = df[num_cols].copy()

# Объединяем всё в финальную матрицу признаков
X = pd.concat([df_num, df_cat], axis=1).fillna(0.0).astype(float)

# Целевая переменная
if price_col is None:
    price_col = 'price_surrogate'
    df[price_col] = df.select_dtypes(include='number').sum(axis=1)
y = df[price_col].astype(float).values

# Разделение на train / val и стандартизация
X_train, X_val, y_train, y_val = train_test_split(X.values, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X.shape, len(y)


## 🧠 Шаг 4 — KNNRegressor (ручной) и сравнение со sklearn

In [ ]:

# Реализация KNN
class KNNRegressor:
    def __init__(self, n_neighbors=K_NEIGHBORS, weights='distance'):
        self.n_neighbors = n_neighbors
        self.weights = weights

    def fit(self, X, y):
        self.X = X.astype(float)
        self.y = y.astype(float)
        return self

    def _predict_point(self, x):
        dists = np.linalg.norm(self.X - x, axis=1)
        idx = np.argpartition(dists, self.n_neighbors)[:self.n_neighbors]
        nn_y = self.y[idx]
        nn_d = dists[idx] + 1e-12
        if self.weights == 'distance':
            w = 1.0 / nn_d
            return np.sum(w * nn_y) / np.sum(w)
        else:
            return np.mean(nn_y)

    def predict(self, X):
        return np.array([self._predict_point(x) for x in X])

# Метрики качества
def eval_regression(y_true, y_pred):
    return {
        "RMSE": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "R2": float(r2_score(y_true, y_pred))
    }

# Ручной KNN
knn_custom = KNNRegressor().fit(X_train, y_train)
pred_knn_custom = knn_custom.predict(X_val)
metrics_knn_custom = eval_regression(y_val, pred_knn_custom)

# KNN из sklearn
knn_sklearn = SKKNN(n_neighbors=K_NEIGHBORS, weights='distance').fit(X_train, y_train)
pred_knn_sk = knn_sklearn.predict(X_val)
metrics_knn_sk = eval_regression(y_val, pred_knn_sk)

pd.DataFrame([
    {"Model": "KNN Custom", **metrics_knn_custom},
    {"Model": "KNN Sklearn", **metrics_knn_sk}
])


## 🧮 Шаг 5 — Linear Regression (SGD / Momentum / AdaGrad) + сравнение со sklearn

In [ ]:

# Реализация линейной регрессии с разными оптимизаторами
class LinearRegressionGD:
    def __init__(self, lr=0.05, epochs=400, optimizer='sgd', momentum=0.9, eps=1e-8, random_state=RANDOM_STATE):
        self.lr = lr
        self.epochs = epochs
        self.optimizer = optimizer
        self.momentum = momentum
        self.eps = eps
        self.random_state = random_state
        self.loss_history_ = []

    def _add_bias(self, X):
        return np.hstack([np.ones((X.shape[0],1)), X])

    def fit(self, X, y):
        rng = np.random.default_rng(self.random_state)
        Xb = self._add_bias(X)
        n, d = Xb.shape
        self.w = rng.normal(scale=0.01, size=d)
        v = np.zeros_like(self.w)
        G = np.zeros_like(self.w)

        for _ in range(self.epochs):
            y_pred = Xb @ self.w
            err = y_pred - y
            grad = (2.0/n) * (Xb.T @ err)

            if self.optimizer == 'sgd':
                update = -self.lr * grad
            elif self.optimizer == 'momentum':
                v = self.momentum * v - self.lr * grad
                update = v
            elif self.optimizer == 'adagrad':
                G += grad**2
                update = - (self.lr / (np.sqrt(G) + self.eps)) * grad
            else:
                raise ValueError("unknown optimizer")

            self.w += update
            self.loss_history_.append(float((err**2).mean()))
        return self

    def predict(self, X):
        Xb = self._add_bias(X)
        return Xb @ self.w

# Обучаем 3 варианта + sklearn
lr_sgd = LinearRegressionGD(lr=0.05, epochs=400, optimizer='sgd').fit(X_train, y_train)
lr_mom = LinearRegressionGD(lr=0.05, epochs=400, optimizer='momentum').fit(X_train, y_train)
lr_ada = LinearRegressionGD(lr=0.2, epochs=400, optimizer='adagrad').fit(X_train, y_train)

pred_lr_sgd = lr_sgd.predict(X_val)
pred_lr_mom = lr_mom.predict(X_val)
pred_lr_ada = lr_ada.predict(X_val)

m_lr_sgd = eval_regression(y_val, pred_lr_sgd)
m_lr_mom = eval_regression(y_val, pred_lr_mom)
m_lr_ada = eval_regression(y_val, pred_lr_ada)

sk_lr = SKLR().fit(X_train, y_train)
m_sk_lr = eval_regression(y_val, sk_lr.predict(X_val))

sk_sgd = SGDRegressor(loss="squared_error", max_iter=1000, tol=1e-3, random_state=RANDOM_STATE).fit(X_train, y_train)
m_sk_sgd = eval_regression(y_val, sk_sgd.predict(X_val))

pd.DataFrame([
    {"Model": "LinReg SGD (custom)", **m_lr_sgd},
    {"Model": "LinReg Momentum (custom)", **m_lr_mom},
    {"Model": "LinReg AdaGrad (custom)", **m_lr_ada},
    {"Model": "LinearRegression (sklearn)", **m_sk_lr},
    {"Model": "SGDRegressor (sklearn)", **m_sk_sgd},
])


## 📉 Шаг 6 — Графики функции потерь

In [ ]:

# Визуализация сходимости
def plot_loss(history, title):
    plt.figure(figsize=(6,4))
    plt.plot(history)
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_loss(lr_sgd.loss_history_, "Linear Regression — SGD")
plot_loss(lr_mom.loss_history_, "Linear Regression — Momentum")
plot_loss(lr_ada.loss_history_, "Linear Regression — AdaGrad")


## 📊 Шаг 7 — Финальная таблица метрик

In [ ]:

def row(name, m): return {"Model":name, **m}
rows = [
    row("KNN (custom)", metrics_knn_custom),
    row("KNN (sklearn)", metrics_knn_sk),
    row("LinReg (SGD custom)", m_lr_sgd),
    row("LinReg (Momentum custom)", m_lr_mom),
    row("LinReg (AdaGrad custom)", m_lr_ada),
    row("LinearRegression (sklearn)", m_sk_lr),
    row("SGDRegressor (sklearn)", m_sk_sgd)
]
metrics_df = pd.DataFrame(rows).sort_values(by="RMSE")
metrics_df



## 📌 Выводы:
- Мы реализовали и протестировали несколько моделей (KNN и Linear Regression) и оптимизаторов.  
- Добавили гео- и временные признаки, категориальные закодировали через OHE.  
- Momentum и AdaGrad сходятся быстрее, чем чистый SGD.  
- KNN дал схожие результаты с `sklearn`.  
- Финальная таблица позволяет сравнить точность всех моделей.
